In [357]:
import pandas as pd
import os
import datetime as dt

In [358]:
filepath = 'data'

## Creating Test Data

In [359]:
current_date = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
current_date

'2024-02-17 17:34:41'

In [360]:
current_date = dt.datetime.now()

In [361]:

date_list = []

for i in range(7):
    date_string = current_date.strftime('%Y-%m-%d_%H-%M-%S')
    date_list.append(date_string)
    current_date += dt.timedelta(days=1)

print(date_list)

['2024-02-17_17-34-41', '2024-02-18_17-34-41', '2024-02-19_17-34-41', '2024-02-20_17-34-41', '2024-02-21_17-34-41', '2024-02-22_17-34-41', '2024-02-23_17-34-41']


In [362]:
name_str = 'Sales_Data'
new_list = [name_str + '_' + date for date in date_list]

In [363]:
new_list

['Sales_Data_2024-02-17_17-34-41',
 'Sales_Data_2024-02-18_17-34-41',
 'Sales_Data_2024-02-19_17-34-41',
 'Sales_Data_2024-02-20_17-34-41',
 'Sales_Data_2024-02-21_17-34-41',
 'Sales_Data_2024-02-22_17-34-41',
 'Sales_Data_2024-02-23_17-34-41']

In [364]:
json_data =[
  {
    "product": "Widget A",
    "quantity": 100,
    "price_per_unit": 10.50,
    "total_sales": 1050.00
  },
  {
    "product": "Widget B",
    "quantity": 50,
    "price_per_unit": 25.75,
    "total_sales": 1287.50
  },
  {
    "product": "Widget C",
    "quantity": 75,
    "price_per_unit": 15.00,
    "total_sales": 1125.00
  },
  {
    "product": "Widget D",
    "quantity": 30,
    "price_per_unit": 50.00,
    "total_sales": 1500.00
  },
  {
    "product": "Widget E",
    "quantity": 20,
    "price_per_unit": 40.25,
    "total_sales": 805.00
  },
  {
    "product": "Widget F",
    "quantity": 60,
    "price_per_unit": 18.99,
    "total_sales": 1139.40
  },
  {
    "product": "Widget G",
    "quantity": 45,
    "price_per_unit": 30.00,
    "total_sales": 1350.00
  }
]



In [365]:
json_data[0]

{'product': 'Widget A',
 'quantity': 100,
 'price_per_unit': 10.5,
 'total_sales': 1050.0}

In [366]:
import csv
for i in range(len(new_list)):
    filename = os.path.splitext(new_list[i])[0] + ".csv"
    with open(os.path.join(filepath, filename), 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=["product", "quantity", "price_per_unit", "total_sales"])
        writer.writeheader()
        writer.writerow(json_data[i])


## Once Data Files are created we now need to make the function to read all these files into a table

In [367]:
dir_files = [f for f in os.listdir(filepath)]

In [368]:
dir_files

['hello.txt',
 'hi.txt',
 'Sales_Data_2024-02-17_17-34-41.csv',
 'Sales_Data_2024-02-18_17-34-41.csv',
 'Sales_Data_2024-02-19_17-34-41.csv',
 'Sales_Data_2024-02-20_17-34-41.csv',
 'Sales_Data_2024-02-21_17-34-41.csv',
 'Sales_Data_2024-02-22_17-34-41.csv',
 'Sales_Data_2024-02-23_17-34-41.csv']

In [369]:
files_to_read = [f for f in os.listdir(filepath) if f.startswith('Sales_Data')]

In [370]:
files_to_read

['Sales_Data_2024-02-17_17-34-41.csv',
 'Sales_Data_2024-02-18_17-34-41.csv',
 'Sales_Data_2024-02-19_17-34-41.csv',
 'Sales_Data_2024-02-20_17-34-41.csv',
 'Sales_Data_2024-02-21_17-34-41.csv',
 'Sales_Data_2024-02-22_17-34-41.csv',
 'Sales_Data_2024-02-23_17-34-41.csv']

In [371]:
df = pd.DataFrame()


In [372]:
type(df)

pandas.core.frame.DataFrame

In [373]:
files_to_read[0].split('_')[2]

'2024-02-17'

In [374]:
def into_table(filepath: str, distinct: set, initial_df: pd.DataFrame) -> pd.DataFrame:
    try:
        files_to_read = [f for f in os.listdir(filepath) if f.startswith('Sales_Data')]
        for file in files_to_read:
            date = file.split('_')[2]
            if date not in distinct:
                df = pd.read_csv(os.path.join(filepath, file))
                df['Date'] = date
                initial_df = pd.concat([initial_df, df], ignore_index=True)
                distinct.add(date)
            else:
                
                initial_df = initial_df[~initial_df['Date'].str.startswith(date)]
                df = pd.read_csv(os.path.join(filepath, file))
                df['Date'] = date
                initial_df = pd.concat([initial_df, df], ignore_index=True)
        
        return initial_df
    except Exception as e:
        print("An error occurred:", e)
        return initial_df


In [375]:
distinct = set()
initial_df = pd.DataFrame()
df = into_table(filepath,distinct,initial_df)

In [376]:
df.reset_index(inplace=True,drop=True)
df

,product,quantity,price_per_unit,total_sales,Date
0,Widget A,100,10.50,1050.0,2024-02-17
1,Widget B,50,25.75,1287.5,2024-02-18
2,Widget C,75,15.00,1125.0,2024-02-19
3,Widget D,30,50.00,1500.0,2024-02-20
4,Widget E,20,40.25,805.0,2024-02-21
5,Widget F,60,18.99,1139.4,2024-02-22
6,Widget G,45,30.00,1350.0,2024-02-23


#### If the name doesnt exists the function will do the following steps :
####    1. It will read the file from the filepath using pandas.read_csv
####    2. It will then add a new column named date to extract the date from the file name to the dataframe
####    3. It will concatinate the initial df which is empty to the df we just read
####    4. It will then enter the date  into the set 'distinct' and return the updated df with added date column
#### However if the name exists it will do the following : 
####    1. Delete the old records of the date
####    2. Read the file again 
####    3. Add the date column again
####    4. Append the data to the initial df once again and return the initial df

#### This would suffice both the conditions provided i.e. :
####    1.The code should only process files that were not read during the last execution
####    2.Include a mechanism to reprocess previously executed files, deleting their records and reloading them

#### However the function would delete the data without checking if it is modified or not for that we can use the time constraints as well and add it as a column

In [377]:
time = (files_to_read[0].split('_')[3].split('.')[0])
formatted_time =  dt.datetime.strptime(time, '%H-%M-%S').strftime('%H:%M:%S')
datetime = files_to_read[0].split('_')[2]+'_'+formatted_time
datetime

'2024-02-17_17:34:41'

In [378]:
time_string = '15-54-00'
time_obj = dt.datetime.strptime(time_string, '%H-%M-%S')
formatted_time = time_obj.strftime('%H:%M:%S')
formatted_time
time_string1 = '15-55-00'
time_obj1 = dt.datetime.strptime(time_string1, '%H-%M-%S')
formatted_time1 = time_obj1.strftime('%H:%M:%S')

print(formatted_time1 > formatted_time)

True


In [379]:
def into_table_modified(filepath: str, distinct: set, initial_df: pd.DataFrame) -> pd.DataFrame:
    try:
        files_to_read = [f for f in os.listdir(filepath) if f.startswith('Sales_Data')]
        for file in files_to_read:
            date = file.split('_')[2]
            time = file.split('_')[3].split('.')[0]
            formatted_time = dt.datetime.strptime(time, '%H-%M-%S').strftime('%H:%M:%S')
            if date not in distinct:
                df = pd.read_csv(os.path.join(filepath, file))
                df['Date'] = date
                df['Time Stamp'] = formatted_time
                initial_df = pd.concat([initial_df, df], ignore_index=True)
                distinct.add(date)
            else:
                max_time_for_date = initial_df.loc[initial_df['Date'] == date, 'Time Stamp'].max()
                if formatted_time > max_time_for_date:
                    initial_df = initial_df[~initial_df['Date'].str.startswith(date)]
                    df = pd.read_csv(os.path.join(filepath, file))
                    df['Date'] = date
                    df['Time Stamp'] = formatted_time
                    initial_df = pd.concat([initial_df, df], ignore_index=True)
        
        return initial_df
    except Exception as e:
        print("An error occurred:", e)
        return initial_df

In [380]:
distinct = set()

In [381]:
initial_df = pd.DataFrame()

In [382]:
df_new = into_table_modified(filepath,distinct,initial_df)

In [383]:
df_new

,product,quantity,price_per_unit,total_sales,Date,Time Stamp
0,Widget A,100,10.50,1050.0,2024-02-17,17:34:41
1,Widget B,50,25.75,1287.5,2024-02-18,17:34:41
2,Widget C,75,15.00,1125.0,2024-02-19,17:34:41
3,Widget D,30,50.00,1500.0,2024-02-20,17:34:41
4,Widget E,20,40.25,805.0,2024-02-21,17:34:41
5,Widget F,60,18.99,1139.4,2024-02-22,17:34:41
6,Widget G,45,30.00,1350.0,2024-02-23,17:34:41


#### Adding a new file with same date but different time stamp



In [384]:
current_date1 = dt.datetime.now()
date_string = current_date1.strftime('%Y-%m-%d_%H-%M-%S')
name = 'Sales_Data'+ '_' + date_string
json =[ {
    "product": "Widget H",
    "quantity": 42,
    "price_per_unit": 31.00,
    "total_sales": 1250.00
  }
]
filename = os.path.splitext(name)[0] + ".csv"
with open(os.path.join(filepath, filename), 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=["product", "quantity", "price_per_unit", "total_sales"])
    writer.writeheader()
    writer.writerow(json[0])



In [385]:
into_table_modified(filepath,distinct,df_new)

,product,quantity,price_per_unit,total_sales,Date,Time Stamp
0,Widget B,50,25.75,1287.5,2024-02-18,17:34:41
1,Widget C,75,15.00,1125.0,2024-02-19,17:34:41
2,Widget D,30,50.00,1500.0,2024-02-20,17:34:41
3,Widget E,20,40.25,805.0,2024-02-21,17:34:41
4,Widget F,60,18.99,1139.4,2024-02-22,17:34:41
5,Widget G,45,30.00,1350.0,2024-02-23,17:34:41
6,Widget H,42,31.00,1250.0,2024-02-17,17:34:42
